##과제:외부 블로그의 정보와 함께 챗봇 구현하기

요구사항: https://www.notion.so/teamsparta/5-1782dc3ef5148036b6d6cd7775c76f25

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dajinfx/hanghaeAI2/blob/main/W5_1_Rag.ipynb)


In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
import openai
import json
import requests
import numpy as np
import torch
import pandas as pd
from langchain.prompts import PromptTemplate

In [4]:
gpt_model="gpt-4o"
temperature = 0.7  # 각 token을 샘플링할 때 사용하는 temperature 값입니다.
max_tokens = 4096  # 생성하는 최대 token 개수 입니다.
frequency_penalty = 0.0  # 같은 단어가 반복적으로 나오는 것을 방지하기 위한 옵션입니다.

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

In [5]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
    print(f" {filename} is uploaded")

with open('OpenAI_APIKey.txt', 'r') as file:
    api_key = file.read()


Saving OpenAI_APIKey.txt to OpenAI_APIKey.txt
 OpenAI_APIKey.txt is uploaded


In [7]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [8]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["editedContent"]},
        )
    ),
    header_template={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    },
    encoding="utf-8",
)
docs = loader.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

In [11]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들 모두 나열하고 요약해줘"
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들 모두 나열하고 요약해줘 \nContext: 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소통 능력을 기를 수 있는 문화의 장으로 쓰일 수 있겠죠!사용한 기술 스택-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket-BE(백엔드): React Native, TanStack Query, Axios코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!👉🏻\xa0공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 

In [12]:
user_prompt

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들 모두 나열하고 요약해줘 \nContext: 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소통 능력을 기를 수 있는 문화의 장으로 쓰일 수 있겠죠!사용한 기술 스택-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket-BE(백엔드): React Native, TanStack Query, Axios코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!👉🏻\xa0공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다

In [13]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전에서 수상한 작품은 다음과 같습니다: 

1. **대상**: [Lexi Note] - 언어공부 필기 웹 서비스
2. **우수상**: [에코 클래스룸] - 수업 실시간 소통 서비스
3. **입선**: [Crewing] - 연합동아리 정보 플랫폼
4. **입선**: [학교생활 매니저] - 학교생활 관리 서비스

각 수상작은 학생들이 직면한 문제를 해결하기 위한 혁신적인 아이디어로 구성되어 있습니다.
